<a href="https://colab.research.google.com/github/KaveeshBaddage/DataScienceImpl/blob/main/Generative_Adversarial_Network_(GAN)_for_MNIST_kaveesha_v4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import required libraries

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.autograd import Variable
from torchvision.utils import save_image
from torchvision import datasets, transforms

import numpy as np

# to save Generator, Discriminator
import pickle as pkl

#to create and download the zipped files
import shutil
from google.colab import files

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Load MNIST train and test data 


In [3]:
# define the data transformer
transform = transforms.Compose([transforms.ToTensor(),
  transforms.Normalize((0.5,), (0.5,))
])

# get train dataset
train_dataset = datasets.MNIST(
    root='data/',
    train=True,
    download=True,
    transform=transform
    )  
print("train_dataset size ->",train_dataset.data.size())

# Dataset stores the samples and their corresponding labels, and DataLoader wraps an iterable around the Dataset to enable easy access to the samples.
train_loader = torch.utils.data.DataLoader(
    dataset=train_dataset, 
    batch_size=100,
    shuffle=True
    # num_workers=0
    )

# get test dataset
test_dataset = datasets.MNIST(
    root='data/', 
    train=False, 
    transform=transform
    )
print("test_dataset size ->",test_dataset.data.size())

#DataLoader wraps an iterable around the test_dataset to enable easy access to the samples.
test_loader = torch.utils.data.DataLoader(
    dataset=test_dataset, 
    batch_size=100,
    # num_workers=0
    shuffle=True
    )


Extracting data/MNIST/raw/train-images-idx3-ubyte.gz to data/MNIST/raw




Extracting data/MNIST/raw/train-labels-idx1-ubyte.gz to data/MNIST/raw




Extracting data/MNIST/raw/t10k-images-idx3-ubyte.gz to data/MNIST/raw




Extracting data/MNIST/raw/t10k-labels-idx1-ubyte.gz to data/MNIST/raw

train_dataset size -> torch.Size([60000, 28, 28])
test_dataset size -> torch.Size([10000, 28, 28])


/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


# Define Discriminator and Generator classes

In [4]:
# defining Discriminator class

class Discriminator(nn.Module):

    def __init__(self, input_size, d_output_size):

        # access methods from parent class nn.Module
        super(Discriminator, self).__init__()
        
        # Hidden linear layers
        self.fc1 = nn.Linear(input_size, 1024)
        self.fc2 = nn.Linear(self.fc1.out_features, self.fc1.out_features//2)
        self.fc3 = nn.Linear(self.fc2.out_features, self.fc1.out_features//2)
        
        # Final output layer
        self.fc4 = nn.Linear(self.fc3.out_features, d_output_size)
        
    def forward(self, x):
        # flatten image
        x = x.view(-1, 28*28) #  reshape the tensor to 28*28 tensor
        # all hidden layers
        x = F.leaky_relu(self.fc1(x), 0.2) # (input, negative_slope=0.2) This function returns x if it receives any positive input, but for any negative value of x, it returns a really small value which is 0.02 times x
        x = F.dropout(x, 0.3) #  “deactivates” few neurons in the neural network randomly in the probability  in order to avoid the problem of overfitting.
        x = F.leaky_relu(self.fc2(x), 0.2)
        x = F.dropout(x, 0.3)
        x = F.leaky_relu(self.fc3(x), 0.2)
        x = F.dropout(x, 0.3)
        # final layer
        output = torch.sigmoid(self.fc4(x)) # squishes any real number into a range between 0 and 1


        return output

In [5]:
# defining generator class 1

class Generator(nn.Module):

    def __init__(self, input_size, g_output_size):
        super(Generator, self).__init__()
        
        # Hidden linear layers
        self.fc1 = nn.Linear(input_size, 256)
        self.fc2 = nn.Linear(self.fc1.out_features, self.fc1.out_features*2)
        self.fc3 = nn.Linear(self.fc2.out_features, self.fc1.out_features*2)
        
        # Final output layer
        self.fc4 = nn.Linear(self.fc3.out_features, g_output_size)

    def forward(self, x):
        # all hidden layers
        x = F.leaky_relu(self.fc1(x), 0.2) # (input, negative_slope=0.2)
        x = F.leaky_relu(self.fc2(x), 0.2)
        x = F.leaky_relu(self.fc3(x), 0.2)
        output = torch.tanh(self.fc4(x)) #Returns a new tensor with the hyperbolic tangent of the elements of self.fc4(x)

        return output

# Define Hyper Parameters and initiate the Discriminator and Generator

In [6]:

# Discriminator hyperparams

# Size of input image to Discriminator (28*28)
input_size = 784
# Size of discriminator output (real or fake)
d_output_size = 1


# Generator hyperparams

#Generator input  latent vector size
z_size = 100
# Discriminator output size
g_output_size = 784



D = Discriminator(input_size, d_output_size).to(device)
G = Generator(z_size, g_output_size).to(device)

print(D)
print(G)

Discriminator(
  (fc1): Linear(in_features=784, out_features=1024, bias=True)
  (fc2): Linear(in_features=1024, out_features=512, bias=True)
  (fc3): Linear(in_features=512, out_features=512, bias=True)
  (fc4): Linear(in_features=512, out_features=1, bias=True)
)
Generator(
  (fc1): Linear(in_features=100, out_features=256, bias=True)
  (fc2): Linear(in_features=256, out_features=512, bias=True)
  (fc3): Linear(in_features=512, out_features=512, bias=True)
  (fc4): Linear(in_features=512, out_features=784, bias=True)
)


In [7]:
# Measures the Binary Cross Entropy between the target and the output
criterion = nn.BCELoss() 

# Set learning rate and optimizers
lr = 0.0002 
G_optimizer = optim.Adam(G.parameters(), lr = lr)
D_optimizer = optim.Adam(D.parameters(), lr = lr)

# Define model training functions

In [8]:
def Train_Discriminator(x):
    
    # clear out the gradients of all Variables since no need to keep the gradient calculation result of the previous batch
    D.zero_grad()

    # train discriminator on real
    x_real, y_real = x.view(-1, 28*28), torch.ones(100, 1)
    x_real, y_real = Variable(x_real.to(device)), Variable(y_real.to(device))

    D_output = D(x_real)
    D_real_loss = criterion(D_output, y_real)
    D_real_score = D_output

    # train discriminator on facke
    z = Variable(torch.randn(100, 100).to(device))
    x_fake, y_fake = G(z), Variable(torch.zeros(100, 1).to(device))

    D_output = D(x_fake)
    D_fake_loss = criterion(D_output, y_fake)
    D_fake_score = D_output

    # gradient backprop & optimize ONLY D's parameters
    D_loss = D_real_loss + D_fake_loss
    D_loss.backward()
    D_optimizer.step()
        
    return  D_loss.data.item()

In [9]:
def Train_Generator(x):
    
    # clear out the gradients of all Variables since no need to keep the gradient calculation result of the previous batch
    G.zero_grad()

    z = Variable(torch.randn(100, 100).to(device))
    y = Variable(torch.ones(100, 1).to(device))

    G_output = G(z)
    D_output = D(G_output)
    G_loss = criterion(D_output, y)

    # gradient backprop & optimize ONLY G's parameters
    G_loss.backward()
    G_optimizer.step()
        
    return G_loss.data.item()

# Train the models

In [10]:
n_epoch = 200
for epoch in range(1, n_epoch+1):           
    D_losses, G_losses = [], []
    for batch_idx, (x, _) in enumerate(train_loader):
        D_losses.append(Train_Discriminator(x))
        G_losses.append(Train_Generator(x))

    print('[%d/%d]: loss_d: %.3f, loss_g: %.3f' % (
            (epoch), n_epoch, torch.mean(torch.FloatTensor(D_losses)), torch.mean(torch.FloatTensor(G_losses))))

[1/200]: loss_d: 0.998, loss_g: 2.149
[2/200]: loss_d: 0.983, loss_g: 1.971
[3/200]: loss_d: 1.045, loss_g: 2.336
[4/200]: loss_d: 0.988, loss_g: 1.666
[5/200]: loss_d: 0.593, loss_g: 2.919
[6/200]: loss_d: 0.363, loss_g: 4.041
[7/200]: loss_d: 0.605, loss_g: 2.985
[8/200]: loss_d: 0.608, loss_g: 2.721
[9/200]: loss_d: 0.544, loss_g: 2.751
[10/200]: loss_d: 0.560, loss_g: 2.656
[11/200]: loss_d: 0.605, loss_g: 2.521
[12/200]: loss_d: 0.663, loss_g: 2.245
[13/200]: loss_d: 0.674, loss_g: 2.210
[14/200]: loss_d: 0.703, loss_g: 2.330
[15/200]: loss_d: 0.700, loss_g: 2.249
[16/200]: loss_d: 0.740, loss_g: 2.145
[17/200]: loss_d: 0.723, loss_g: 2.051
[18/200]: loss_d: 0.750, loss_g: 2.039
[19/200]: loss_d: 0.784, loss_g: 1.954
[20/200]: loss_d: 0.774, loss_g: 1.945
[21/200]: loss_d: 0.817, loss_g: 1.862
[22/200]: loss_d: 0.840, loss_g: 1.767
[23/200]: loss_d: 0.823, loss_g: 1.826
[24/200]: loss_d: 0.853, loss_g: 1.761
[25/200]: loss_d: 0.854, loss_g: 1.685
[26/200]: loss_d: 0.909, loss_g: 1

# Save and download the models

In [11]:
# Save the Generator
with open('G.pkl', 'wb') as f:
    pkl.dump(G, f)

# Save the Discriminator
with open('D.pkl', 'wb') as f:
    pkl.dump(D, f)


# download files into the local machine
files.download('D.pkl')
files.download('G.pkl')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Create and annotate a fake data set

In [22]:
#create directory to save generated images and files
!mkdir images

#create 100 latent vectors which has 100  dimension

sample_size=100

z = np.random.uniform(-1, 1, size=(sample_size, z_size))
z = torch.from_numpy(z).float()

G.eval() # eval mode

# set image directory root location
root = '/content'

#get the grid image
grid_view = G(z.to(device))
save_image(grid_view.view(grid_view.size(0), 1, 28, 28), root + r'/grid.png', normalize = True) 


for i, image_tensor in enumerate(z):
    image_tensor = image_tensor.detach()

    # save the latent vector, related to the image
    f = open(root + r'/images/%d.txt' %i,'a')
    f.write(np.array2string(image_tensor.numpy(), separator=','))
    f.close()

    # create image using latent vector
    rand_image = G(image_tensor.to(device)) 

    # save image
    rand_image = rand_image.view(-1, 28) # convert [784] tensor in to [28,28] tensor to create image
    save_image(rand_image, root + r'/images/%d.png' %i, normalize = True)


#create and download zip file contains generated image files and Latent vectors
shutil.make_archive('images', 'zip', 'images')
files.download('images.zip')

'/content/images.zip'

# Create random digit images From the real MNIST test data set

In [23]:
#create directory to save generated images and test files
!mkdir real_images

# extract 100 random digit images from the real MNIST test data set

for idx, (img, _) in enumerate(test_dataset):

    # save the latent vector, related to the image
    f = open(root + r'/real_images/%d.txt' %idx,'a')
    f.write(np.array2string(img.numpy(), separator=','))
    f.close()

    # save the image
    save_image(img, '/content/real_images/%d.png' %idx, normalize = True)

    # stop the iteration after creating 100 images
    if idx == 100: 
        break   

#create and download a zip file contains generated image files and Latent vectors
shutil.make_archive('real_images', 'zip', 'real_images')
files.download('real_images.zip')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
#unzip the uploaded s0 zipped file
!unzip '/content/Fake_Digits.zip' -d '/content/Fake_Digits'

In [ ]:
#open previously created Generator
with open('G.pkl', 'rb') as f:
    G = pkl.load(f)

# Create Classifier

In [ ]:
3upload image zip files and extract them
!unzip '/content/real_images.zip' -d '/content/real_images'
!unzip '/content/Fake_Digits.zip' -d '/content/Fake_Digits'

In [ ]:
data_dir = '/content/Fake_Digits/Fake_Digits/'

#create train and test dataset using images

def load_split_train_test(datadir, valid_size = .2):
    train_transforms = transforms.Compose([transforms.Resize(224),
                                       transforms.ToTensor(),
                                       ])
    test_transforms = transforms.Compose([transforms.Resize(224),
                                      transforms.ToTensor(),
                                      ])
    train_data = datasets.ImageFolder(datadir,       
                    transform=train_transforms)
    test_data = datasets.ImageFolder(datadir,
                    transform=test_transforms)
    num_train = len(train_data)
    indices = list(range(num_train))
    split = int(np.floor(valid_size * num_train))
    np.random.shuffle(indices)
    from torch.utils.data.sampler import SubsetRandomSampler
    train_idx, test_idx = indices[split:], indices[:split]
    train_sampler = SubsetRandomSampler(train_idx)
    test_sampler = SubsetRandomSampler(test_idx)
    trainloader = torch.utils.data.DataLoader(train_data,
                   sampler=train_sampler, batch_size=64)
    testloader = torch.utils.data.DataLoader(test_data,
                   sampler=test_sampler, batch_size=64)
    return trainloader, testloader
trainloader, testloader = load_split_train_test(data_dir, .2)

print(trainloader.dataset.classes)
